# Python Assignment - Question 1
## Assumptions made:
Data in excel A and excel b are identical, so made an assumption that same order ids can exist in different excels, else while removing duplicates one excel would completely get removed.

Business Rules:
1. Combine the data from both regions into a single table. 
2. Add a column total_sales which is calculated as QuantityOrdered * ItemPrice.
3. Add a column region to identify the region of the sales record (A or B).
4. Ensure that there are no duplicate entries based on OrderId.
5. Add a new column net_sale, calculated as total_sales - PromotionDiscount.
6. Exclude orders where the total sales amount is negative or zero after applying discounts.
7. Load the transformed data into a the database of your choice.

First the business rules transformations are applied on both excels separately and then both regions are combined into a single table as per business rule 1. Duplicate orderIds are removed from each excel separately.

In [ ]:
# importing required libraries
import pandas as pd
import sqlite3
import json

### TASK1

In [72]:
# Function to extract data from CSV
def extract_data(file_path):
    return pd.read_csv(file_path)

df = extract_data(file_path)

### TASK2

In [73]:
def transform_data(df, region):
    
    df = df.copy()
#   Calculating the total sales amount
    df['total_sales'] = df['QuantityOrdered'] * df['ItemPrice']
    
#   Adding a column to identify the region
    df['region'] = region
    
#   Ensure unique entries based on OrderId
    df = df.drop_duplicates(subset=["OrderId"]).copy()
    
#   Function for extracting the discount value from Promotion discount    
    def extract_discount(value):
        try:
            if pd.isna(value) or value == "":  
                return 0.0
            discount_dict = json.loads(value)  
            return float(discount_dict.get("Amount", 0)) 
        except (json.JSONDecodeError, TypeError, ValueError):
            return 0.0 

#   Extracting promotion discount and calculating net_sale        
    df.loc[:, "PromotionDiscount"] = df["PromotionDiscount"].apply(extract_discount)
    
    df.loc[:, "net_sale"] = df["total_sales"] - df["PromotionDiscount"]
    
#   Excluding the orders whose total sales amount is negative or zero    
    df = df[df["total_sales"] > 0].copy()
    return df

### TASK3

In [74]:
# Function to load the data to the db

def load_data_to_db(df, db_name='sales_data.db'):
    conn = sqlite3.connect(db_name)
    df.to_sql('sales_data', conn, if_exists='replace', index=False)
    conn.close()

In [75]:
def etl_process():
    # Extracting data from csv
    df_a = extract_data('C:/Users/ajaya/Documents/order_region_a(in).csv')
    df_b = extract_data('C:/Users/ajaya/Documents/order_region_b(in).csv')
    
    # Transforming the data
    df_a = transform_data(df_a, 'A')
    df_b = transform_data(df_b, 'B')
    
    # Combining  data
    df_comb = pd.concat([df_a, df_b], ignore_index=True)
    
#     record_count = df_comb.shape[0]
#     print("Number of records:", record_count)
    
    #Load
    load_data_to_db(df_comb)


In [76]:
etl_process()

Number of records: 41107
Number of records: 41107
Number of records: 82104


### TASK4 - Python functions to validate data

In [85]:
# Database connection
DB_PATH = "sales_data.db"

""" Count total records in the sales_data table """
def get_total_records():
    conn = sqlite3.connect(DB_PATH)
    query = "SELECT COUNT(*) FROM sales_data"
    total_records = pd.read_sql(query, conn).iloc[0, 0]
    conn.close()
    return total_records


""" Get total sales amount by region """
def get_total_sales_by_region():
    conn = sqlite3.connect(DB_PATH)
    query = "SELECT region, SUM(net_sale) AS total_sales FROM sales_data GROUP BY region"
    df = pd.read_sql(query, conn)
    conn.close()
    df["total_sales"] = df["total_sales"].apply(lambda x: "{:,.2f}".format(x))
    return df


""" Get the average sales amount per transaction """
def get_avg_sales_per_transaction():
    conn = sqlite3.connect(DB_PATH)
    query = "SELECT AVG(net_sale) AS avg_sales FROM sales_data"
    avg_sales = pd.read_sql(query, conn).iloc[0, 0]
    conn.close()
    return avg_sales


""" Check for duplicate OrderId values """
def check_duplicate_order_ids():
    conn = sqlite3.connect(DB_PATH)
    query = "SELECT OrderId, COUNT(*) AS count FROM sales_data GROUP BY region, OrderId HAVING COUNT(*) > 1"
    df_duplicates = pd.read_sql(query, conn)
    conn.close()
    return df_duplicates

# Printing output
print("Total Records:", get_total_records())
print("\nTotal Sales by Region:\n",get_total_sales_by_region())
print("\nAverage Sales per Transaction:", get_avg_sales_per_transaction())
print("\nDuplicate Order IDs:\n", check_duplicate_order_ids())


Total Records: 82104

Total Sales by Region:
   region    total_sales
0      A  34,158,411.02
1      B  34,158,411.02

Average Sales per Transaction: 832.0766593584049

Duplicate Order IDs:
 Empty DataFrame
Columns: [OrderId, count]
Index: []


### TASK4 SQL queries to validate data

SELECT COUNT(*) AS total_records FROM sales_data;

SELECT region, SUM(net_sale) AS total_sales
FROM sales_data
GROUP BY region;

SELECT AVG(net_sale) AS avg_sales_per_transaction FROM sales_data;

SELECT OrderId, COUNT(*) AS duplicate_count
FROM sales_data
GROUP BY region,OrderId
HAVING COUNT(*) > 1;